In [2]:
!pip install -q gradio>=4.22.0 pandas langchain langchain-groq chromadb sentence-transformers
!pip install -q datasets spacy python-dotenv accelerate peft bitsandbytes
!python -m spacy download en_core_web_sm
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [3]:
# %% [Import Libraries]
import os
import time
import spacy
import random
import logging
import numpy as np
import pandas as pd
import torch
import chromadb
import gradio as gr
from tqdm import tqdm
from datasets import load_dataset
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

In [4]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# %% [Sentiment Analysis Class]
class SentimentAnalyzer:
    def __init__(self, use_mock=False):
        """
        Initialize the sentiment analyzer
        Args:
            use_mock: If True, use a mock model instead of loading BERT (for testing/demo)
        """
        self.use_mock = use_mock
        self.labels = ["anger", "sadness", "anxiety", "joy", "neutral"]

        if not use_mock:
            try:
                logger.info("Loading sentiment analysis model...")
                self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
                # Using a smaller model (distilbert) for faster inference
                self.model = AutoModelForSequenceClassification.from_pretrained(
                    "distilbert-base-uncased",
                    num_labels=len(self.labels)
                )

                # Move model to GPU if available
                self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                self.model.to(self.device)
                logger.info(f"Model loaded and moved to {self.device}")
            except Exception as e:
                logger.error(f"Error loading sentiment model: {str(e)}")
                logger.info("Falling back to mock sentiment analyzer")
                self.use_mock = True

    def predict_mood(self, text):
        """
        Predict the mood/sentiment from text
        """
        if self.use_mock:
            # For testing or when model loading fails, use random sentiment with weighted distribution
            weights = [0.1, 0.2, 0.2, 0.25, 0.25]  # More neutral and joy, less anger
            return np.random.choice(self.labels, p=weights)

        try:
            # Actual sentiment prediction logic
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = self.model(**inputs)

            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_label_index = torch.argmax(probs).item()
            return self.labels[predicted_label_index]
        except Exception as e:
            logger.error(f"Error in sentiment prediction: {str(e)}")
            return "neutral"  # Default fallback


In [5]:
# %% [Text Preprocessing]
def load_spacy_model():
    """Load spaCy model with caching to prevent reloading"""
    try:
        return spacy.load("en_core_web_sm", disable=["parser", "ner"])
    except OSError:
        # If model not found, download it
        os.system("python -m spacy download en_core_web_sm")
        return spacy.load("en_core_web_sm", disable=["parser", "ner"])

nlp = load_spacy_model()

def preprocess_text(text):
    """Clean and normalize text"""
    if not text or not isinstance(text, str):
        return ""

    try:
        doc = nlp(text)
        # Basic preprocessing: lowercase, remove stopwords, keep alpha, min length 2
        result = " ".join([
            token.lemma_.lower() for token in doc
            if not token.is_stop and token.is_alpha and len(token) > 2
        ])
        return result
    except Exception as e:
        logger.error(f"Error preprocessing text: {str(e)}")
        return text.lower()  # Fallback to simple lowercase


In [6]:
# %% [Knowledge Base Setup]
class KnowledgeBase:
    def __init__(self, use_mock=False):
        """
        Initialize the knowledge base
        Args:
            use_mock: If True, use a mock knowledge base instead of actual embeddings
        """
        self.use_mock = use_mock

        if not use_mock:
            try:
                logger.info("Setting up knowledge base...")
                # Use a lightweight embedding model
                self.embeddings = HuggingFaceEmbeddings(
                    model_name="sentence-transformers/all-MiniLM-L6-v2"
                )

                # Use persistent storage with a reasonable path
                db_path = os.path.join(os.getcwd(), "mindmate_db")
                os.makedirs(db_path, exist_ok=True)

                self.chroma_client = chromadb.PersistentClient(path=db_path)

                self.vector_store = Chroma(
                    collection_name="mental_health_knowledge",
                    embedding_function=self.embeddings,
                    client=self.chroma_client
                )
                logger.info(f"Knowledge base initialized at {db_path}")

                # Check if we have documents already
                if self.vector_store._collection.count() == 0:
                    logger.info("No documents found. Loading dataset...")
                    self.load_initial_data()
            except Exception as e:
                logger.error(f"Error setting up knowledge base: {str(e)}")
                logger.info("Falling back to mock knowledge base")
                self.use_mock = True

    def load_initial_data(self):
        """Load initial data into the knowledge base"""
        try:
            # Load a small subset of the dataset for demonstration
            dataset = load_dataset("empathetic_dialogues")
            train_data = dataset["train"]

            # Convert to DataFrame for easier processing
            sample_size = min(2000, len(train_data))  # Limit sample size
            df = pd.DataFrame(train_data[:sample_size])

            # Process and add data in batches to avoid memory issues
            batch_size = 200
            for i in range(0, len(df), batch_size):
                batch = df.iloc[i:i+batch_size]

                # Combine context and utterance for more meaningful vectors
                texts = [
                    f"Context: {row['context']} Response: {row['utterance']}"
                    for _, row in batch.iterrows()
                ]

                # Preprocess texts
                processed_texts = [preprocess_text(text) for text in texts]

                # Add to vector store
                self.vector_store.add_texts(processed_texts)
                logger.info(f"Added batch {i//batch_size + 1} to knowledge base")

            logger.info(f"Successfully loaded {sample_size} examples into knowledge base")
        except Exception as e:
            logger.error(f"Error loading initial data: {str(e)}")

    def get_relevant_context(self, query, k=2):
        """Get relevant context from the knowledge base"""
        if self.use_mock:
            # Mock responses for testing
            contexts = [
                "Dealing with anxiety involves breathing exercises and mindfulness.",
                "Depression often manifests as persistent sadness and loss of interest.",
                "Maintaining social connections is important for mental health.",
                "Regular exercise can improve mood and reduce stress."
            ]
            return random.sample(contexts, min(2, len(contexts)))

        try:
            # Real similarity search
            docs = self.vector_store.similarity_search(query, k=k)
            return [doc.page_content for doc in docs]
        except Exception as e:
            logger.error(f"Error searching knowledge base: {str(e)}")
            return ["I'm here to listen and help with your concerns."]


In [7]:
# %% [LLM Interface]
class LLMInterface:
    def __init__(self, api_key=None, use_mock=False):
        """
        Initialize the LLM interface
        Args:
            api_key: API key for the LLM service
            use_mock: If True, use mock responses instead of actual LLM
        """
        self.use_mock = use_mock
        self.api_key = api_key

        # Define prompt template
        self.prompt_template = PromptTemplate(
            input_variables=["context", "message", "sentiment", "history"],
            template="""
            [SYSTEM] You are MindMate, a compassionate mental health assistant. Your goal is to provide supportive, empathetic responses based on the user's emotional state and message content. Current detected emotion: {sentiment}

            Relevant context from knowledge base: {context}

            Previous conversation: {history}

            Always maintain a warm, supportive tone. If the user expresses severe distress, provide appropriate resources. Keep responses concise (3-5 sentences) and focused on the user's needs.

            [USER] {message}

            [ASSISTANT]
            """
        )

        if not use_mock:
            try:
                # Initialize LLM with API key
                if not api_key:
                    # Try to get from environment
                    api_key = os.environ.get("GROQ_API_KEY")

                if api_key:
                    self.llm = ChatGroq(
                        temperature=0.7,
                        model_name="mixtral-8x7b-32768",
                        groq_api_key=api_key
                    )
                    logger.info("LLM initialized successfully")
                else:
                    logger.warning("No API key found, falling back to mock responses")
                    self.use_mock = True
            except Exception as e:
                logger.error(f"Error initializing LLM: {str(e)}")
                self.use_mock = True

    def generate_response(self, user_input, sentiment, context, history=[]):
        """Generate a response from the LLM"""
        if self.use_mock:
            # Mock responses based on sentiment
            responses = {
                "anger": [
                    "I understand you're feeling frustrated. It's normal to feel this way sometimes. Would you like to talk more about what's causing these feelings?",
                    "I can sense your anger. Taking deep breaths might help in the moment. Would you like to explore some coping strategies together?"
                ],
                "sadness": [
                    "I'm sorry you're feeling down. It's okay to have these emotions. Is there something specific that triggered these feelings?",
                    "I hear that you're going through a difficult time. Remember that it's okay to give yourself space to feel. What small thing might bring you a moment of peace today?"
                ],
                "anxiety": [
                    "Anxiety can be really challenging. Try focusing on your breathing for a moment. What specific concerns are on your mind right now?",
                    "I understand that feeling anxious can be overwhelming. You're not alone in this. Would it help to talk about your specific worries?"
                ],
                "joy": [
                    "It's wonderful to hear you're feeling positive! These moments are worth celebrating. What has contributed to this good feeling?",
                    "I'm glad you're in good spirits. Positive emotions are important to acknowledge. How can we build on this positive energy?"
                ],
                "neutral": [
                    "Thank you for sharing. How else have you been feeling lately?",
                    "I appreciate you reaching out. Is there anything specific on your mind that you'd like to explore further?"
                ]
            }
            return random.choice(responses.get(sentiment, responses["neutral"]))

        try:
            # Prepare conversation history context
            history_text = ""
            if history:
                history_text = "\n".join([
                    f"User: {h['content']}" if h['role'] == 'user' else f"Assistant: {h['content']}"
                    for h in history[-4:]  # Last 4 messages for context
                ])

            # Format context
            context_text = "\n".join(context)

            # Create LLM chain and run
            chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
            response = chain.run(
                context=context_text,
                message=user_input,
                sentiment=sentiment,
                history=history_text
            )

            return response.strip()
        except Exception as e:
            logger.error(f"Error generating response: {str(e)}")
            return "I'm here to listen and support you. Could you tell me more about how you're feeling?"


In [8]:
# %% [Main Application Class]
class MindMate:
    def __init__(self, api_key=None, use_mock=False):
        """
        Initialize the MindMate application
        Args:
            api_key: API key for the LLM service
            use_mock: If True, use mock components for testing/demo
        """
        logger.info("Initializing MindMate components...")
        self.use_mock = use_mock
        self.sentiment_analyzer = SentimentAnalyzer(use_mock=use_mock)
        self.knowledge_base = KnowledgeBase(use_mock=use_mock)
        self.llm_interface = LLMInterface(api_key=api_key, use_mock=use_mock)

        # Crisis detection
        self.crisis_keywords = {
            "suicide", "kill myself", "self harm", "end my life",
            "don't want to live", "hurt myself", "abuse", "die"
        }

        self.crisis_resources = """
        🚨 I notice you mentioned something concerning. If you're in crisis, please reach out for immediate support:

        • National Suicide Prevention Lifeline (US): 1-800-273-8255
        • Crisis Text Line: Text HOME to 741741
        • International Association for Suicide Prevention: https://www.iasp.info/resources/Crisis_Centres/
        • Emergency Services: Call your local emergency number (911, 112, 999)

        Remember, trained professionals are available 24/7 to help. You're not alone.
        """

        logger.info("MindMate initialization complete")

    def process_message(self, user_input, history=None):
        """
        Process user message and generate response
        Args:
            user_input: The user's message
            history: Previous conversation history
        Returns:
            response: The assistant's response
            sentiment: Detected sentiment
        """
        if history is None:
            history = []

        try:
            # Check for crisis keywords
            if any(kw in user_input.lower() for kw in self.crisis_keywords):
                return self.crisis_resources, "crisis"

            # Detect sentiment
            sentiment = self.sentiment_analyzer.predict_mood(user_input)
            logger.info(f"Detected sentiment: {sentiment}")

            # Get relevant context
            context = self.knowledge_base.get_relevant_context(user_input)

            # Generate response
            response = self.llm_interface.generate_response(
                user_input, sentiment, context, history
            )

            return response, sentiment
        except Exception as e:
            logger.error(f"Error processing message: {str(e)}")
            return "I'm here to help. Could you share more about how you're feeling?", "neutral"


In [9]:
from google.colab import userdata
userdata.get('GROQ_API_KEY')

'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'

In [10]:
# %% [Gradio Interface]
def create_gradio_interface(api_key=None, use_mock=False):
    """Create and configure the Gradio interface"""
    # Initialize MindMate
    mindmate = MindMate(api_key=api_key, use_mock=use_mock)

    # Track emotion history for visualization
    emotion_history = []

    def process_message(message, history, session):
        """Handler for processing messages in Gradio"""
        if not message:
            return {
                chatbot: history,
                emotion_graph: None,
                emotion_summary: None,
                error_html: "",
                input_box: ""
            }

        try:
            # Convert history to format expected by MindMate
            formatted_history = [
                {"role": "user" if i % 2 == 0 else "assistant", "content": msg}
                for i, msg in enumerate([m for h in history for m in h])
            ]

            # Process message
            response, sentiment = mindmate.process_message(message, formatted_history)

            # Update emotion history
            session["mood_history"].append(sentiment)
            emotion_history = session["mood_history"]

            # Create plot data
            plot_data = pd.DataFrame({
                "Time": range(len(emotion_history)),
                "Mood": emotion_history
            })

            # Create emotion summary
            if emotion_history:
                emotion_counts = {}
                for emotion in emotion_history:
                    if emotion in emotion_counts:
                        emotion_counts[emotion] += 1
                    else:
                        emotion_counts[emotion] = 1

                emotion_df = pd.DataFrame(
                    {"Emotion": list(emotion_counts.keys()),
                     "Count": list(emotion_counts.values())}
                )
            else:
                emotion_df = pd.DataFrame(columns=["Emotion", "Count"])

            # Format messages correctly
            new_history = history + [[message, response]]

            return {
                chatbot: new_history,
                emotion_graph: plot_data,
                emotion_summary: emotion_df,
                error_html: "",
                input_box: ""  # Clear input box
            }
        except Exception as e:
            logger.error(f"Error in Gradio interface: {str(e)}")
            return {
                error_html: f"<div style='color: red; padding: 10px;'>Error: {str(e)}</div>",
                chatbot: history,
                emotion_graph: None,
                emotion_summary: None,
                input_box: message  # Keep message in case user wants to try again
            }

    def clear_history():
        """Handler for clearing chat history"""
        empty_plot = pd.DataFrame(columns=["Time", "Mood"])
        empty_summary = pd.DataFrame(columns=["Emotion", "Count"])
        return {
            session_state: {"mood_history": []},
            chatbot: [],
            emotion_graph: empty_plot,
            emotion_summary: empty_summary,
            error_html: ""
        }

    with gr.Blocks(theme=gr.themes.Soft(), title="MindMate Pro") as demo:
        # Header Section
        gr.Markdown("""
        # 🧠 MindMate Pro - Advanced Mental Health Assistant
        *Your 24/7 AI-powered mental health companion*
         Created by Wasim Iqbal
        """)

        # Main Chat Interface
        with gr.Row():
            # Left Panel - Chat Interface
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(
                    label="Conversation History",
                    avatar_images=(None, "https://cdn-icons-png.flaticon.com/512/9243/9243155.png"),
                    height=500,
                   # bubble=True,
                    show_copy_button=True
                )
                with gr.Row():
                    input_box = gr.Textbox(
                        placeholder="How are you feeling today?",
                        label="Your Message",
                        lines=2
                    )
                    submit_btn = gr.Button("Send", variant="primary")

                clear_btn = gr.Button("Clear History")

                gr.Examples(
                    examples=[
                        ["I've been feeling really anxious about my upcoming presentation"],
                        ["I can't sleep at night, my mind keeps racing with worries"],
                        ["I feel so happy today, everything is going well!"],
                        ["I'm struggling with my relationships, I feel lonely"]
                    ],
                    inputs=input_box,
                    label="Common Concerns"
                )

            # Right Panel - Analytics
            with gr.Column(scale=2):
                with gr.Tab("Emotion Analysis"):
                    emotion_graph = gr.LinePlot(
                        label="Mood History",
                        show_label=True,
                        interactive=False,
                        x="Time",
                        y="Mood",
                        title="Emotional Journey",
                        height=300
                    )
                    emotion_summary = gr.DataFrame(
                        headers=["Emotion", "Count"],
                        label="Emotion Distribution",
                        interactive=False
                    )

                with gr.Tab("Safety Tools"):
                    gr.Markdown("""
                    **Immediate Help Resources:**
                    - National Suicide Prevention Lifeline (US): 1-800-273-8255
                    - Crisis Text Line: Text HOME to 741741
                    - International Association for Suicide Prevention: [Find a Crisis Center](https://www.iasp.info/resources/Crisis_Centres/)
                    - Emergency Services: Call your local emergency number (911, 112, 999)

                    **Self-care Techniques:**
                    - Deep breathing exercises
                    - Mindfulness meditation
                    - Physical activity
                    - Reaching out to supportive friends or family
                    """)

        # Hidden Components
        session_state = gr.State(value={"mood_history": []})
        error_html = gr.HTML()

        # Component Links
        submit_event = submit_btn.click(
            process_message,
            [input_box, chatbot, session_state],
            [chatbot, emotion_graph, emotion_summary, error_html, input_box]
        )

        input_box.submit(
            process_message,
            [input_box, chatbot, session_state],
            [chatbot, emotion_graph, emotion_summary, error_html, input_box]
        )

        clear_btn.click(
            clear_history,
            outputs=[session_state, chatbot, emotion_graph, emotion_summary, error_html]
        )

    return demo

# %% [Launch Function]
def launch_mindmate(api_key=None, use_mock=False, interface_type="gradio"):
    """
    Launch MindMate in the specified interface
    Args:
        api_key: API key for LLM service
        use_mock: Whether to use mock components (for testing)
        interface_type: "gradio" or "cli" for command-line interface
    """
    if interface_type == "gradio":
        logger.info("Launching Gradio interface...")
        demo = create_gradio_interface(api_key=api_key, use_mock=use_mock)
        demo.launch(server_port=None, share=True)
    else:
        # CLI interface
        logger.info("Launching CLI interface...")
        mindmate = MindMate(api_key=api_key, use_mock=use_mock)

        print("\n🧠 MindMate Pro - Advanced Mental Health Assistant")
        print("Type 'exit' to end the conversation.\n")

        history = []
        while True:
            try:
                user_input = input("\nYou: ")
                if user_input.lower() in ["exit", "quit", "bye"]:
                    print("\nMindMate: Take care. I'm here whenever you need to talk.")
                    break

                response, sentiment = mindmate.process_message(user_input, history)
                history.append({"role": "user", "content": user_input})
                history.append({"role": "assistant", "content": response})

                print(f"\nMindMate: {response}")
                print(f"[Detected emotion: {sentiment}]")

            except KeyboardInterrupt:
                print("\n\nChat session ended. Take care!")
                break
            except Exception as e:
                logger.error(f"Error in CLI: {str(e)}")
                print("\nMindMate: I'm sorry, I encountered an error. Let's continue our conversation.")

# %% [Main Execution]
if __name__ == "__main__":
    # Define your API key directly
    my_api_key = "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"

    # Set use_mock to False explicitly since we have an API key
    use_mock = False

    # Launch with our API key and mock mode disabled
    launch_mindmate(api_key=my_api_key, use_mock=use_mock, interface_type="gradio")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

<ipython-input-10-f4be4247a1fe>:101: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57d0ec3ebf17e52e13.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
